# Setup

In [ ]:
# %pip install -r requirements.txt

# Environment Variables

In [1]:
# Get environment variables

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Utilities

In [2]:
import utils

# Test 1a - OpenAI and Qdrant

In [4]:
# Build chain using OpenAI and Qdrant

embeddings = utils.create_embeddings_openai()

docs = utils.process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)

chunks = utils.chunk_docs_recursive(docs=docs)

print(f'\nNumber of chunks = {len(chunks)}\n')

vector_store = utils.create_qdrant_vector_store(':memory:', 
                                          'holiday-test', 
                                          1536, 
                                          embeddings, 
                                          chunks)

retriever = utils.create_retriever_qdrant(vector_store)

chat_prompt_template = utils.create_chat_prompt_template()

chain = utils.create_chain_openai('gpt-4o', 
                     chat_prompt_template, 
                     retriever)

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.30it/s]



Number of chunks = 2344



In [5]:
# Test the chain with a few questions 

questions = ["What is the annual revenue of Uber?",
"What is the annual revenue of Lyft?",
"How does Uber's revenue compare to Lyft's revenue?",]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)
	print("\n*****")

What is the annual revenue of Uber?
{'response': AIMessage(content='The annual revenue for Uber Technologies, Inc. for the years 2022 and 2023 are as follows:\n\n- **2022:** $31,877 million\n- **2023:** $37,281 million\n\nThese figures can be found in the income statement section of the 10K reports.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 1347, 'total_tokens': 1410, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7d65f1a5b', 'finish_reason': 'stop', 'logprobs': None}, id='run-59e63f8b-ec7d-4999-8c08-d34745f3bf93-0', usage_metadata={'input_tokens': 1347, 'output_tokens': 63, 'total_tokens': 1410, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning

In [6]:
# Evaluate the chain using Ragas

import time

import pandas as pd
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness)

# Get the questions and groundtruths from the dataframe
testset_df = pd.read_csv("testsets/10k_testset.csv")

questions = testset_df["user_input"].values.tolist()
questions = [str(question) for question in questions]

groundtruths = testset_df["reference"].values.tolist()
groundtruths = [str(ground_truth) for ground_truth in groundtruths]

eval_metrics = [
	answer_correctness,
	answer_relevancy,
	context_precision,
	context_recall,
	faithfulness
]

ragas_results, ragas_results_df = utils.run_ragas_evaluation(chain, 
														questions, 
														groundtruths, 
														eval_metrics)

# Write the results to disk
timestr = time.strftime("%Y%m%d%H%M%S")
ragas_results_df.to_csv(f"evaluations/10x_test1_testset_evaluation_{timestr}.csv")

# Show the resutls
print(ragas_results)

What kind of information does the Form 10-K provide about a company's future performance and risks?
{'response': AIMessage(content='The Form 10-K provides information on a company\'s future performance and risks primarily through two components: forward-looking statements and the "Risk Factors" section.\n\n1. **Forward-Looking Statements**: \n   - These statements are designed to predict or illuminate future events related to the company’s financial or operational performance. They include projections about future results, business strategy, management plans, and financial conditions. You can typically identify forward-looking statements by specific phrases including "may," "will," "expect," "anticipate," "intend," "plan," "believe," and others indicating future expectations.\n\n2. **Risk Factors**: \n   - This section outlines the primary risks and uncertainties that could significantly affect the company\'s future performance. It highlights potential challenges and obstacles that the

Evaluating:   0%|          | 0/255 [00:00<?, ?it/s]

Exception raised in Job[29]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[24]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[14]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[4]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[60]: AttributeError('StringIO' object has no attribute 'TP')
Exception raised in Job[70]: AttributeError('StringIO' object has no attribute 'TP')
Exception raised in Job[65]: AttributeError('StringIO' object has no attribute 'TP')
Exception raised in Job[114]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[195]: AttributeError('StringIO' object has no attribute 'sentences')
Exception raised in Job[190]: AttributeError('StringIO' object has no attribute 'sentences')
Exception raised in Job[204]: AttributeError('StringIO' object has no attribute 'sentences')


{'answer_correctness': 0.5977, 'answer_relevancy': 0.7393, 'context_precision': 0.8121, 'context_recall': 0.5047, 'faithfulness': 0.7491}
